In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, ShuffleSplit, GridSearchCV
from sklearn.metrics import roc_auc_score, roc_curve, auc

import os
import time
import sys
import pprint
import xlrd

In [2]:
book = xlrd.open_workbook('Book1.xlsx')
sheet = book.sheet_by_index(0)
keep_cols = []
for i in range(0, sheet.nrows):
    keep_cols.append(sheet.cell(i, 0).value)

In [3]:
df = pd.read_csv('application_train.csv')

In [4]:
for i in range(len(keep_cols)):
    del df[keep_cols[i]]

In [5]:
def convertNegatives(num):
    num = int(num * -1)
    return num
df['AGE'] = df['DAYS_BIRTH'].apply(convertNegatives)
df['TIME_EMP'] = df['DAYS_EMPLOYED'].apply(convertNegatives)

In [6]:
df.shape

(307511, 26)

In [7]:
df.dtypes

SK_ID_CURR                       int64
TARGET                           int64
NAME_CONTRACT_TYPE              object
CODE_GENDER                     object
FLAG_OWN_CAR                    object
FLAG_OWN_REALTY                 object
CNT_CHILDREN                     int64
AMT_INCOME_TOTAL               float64
AMT_CREDIT                     float64
AMT_ANNUITY                    float64
AMT_GOODS_PRICE                float64
NAME_EDUCATION_TYPE             object
NAME_FAMILY_STATUS              object
NAME_HOUSING_TYPE               object
REGION_POPULATION_RELATIVE     float64
DAYS_BIRTH                       int64
DAYS_EMPLOYED                    int64
OCCUPATION_TYPE                 object
CNT_FAM_MEMBERS                float64
REGION_RATING_CLIENT             int64
REGION_RATING_CLIENT_W_CITY      int64
EXT_SOURCE_1                   float64
EXT_SOURCE_2                   float64
EXT_SOURCE_3                   float64
AGE                              int64
TIME_EMP                 

In [8]:
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,DAYS_EMPLOYED,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,AGE,TIME_EMP
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,-637,Laborers,1.0,2,2,0.083037,0.262949,0.139376,9461,637
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,-1188,Core staff,2.0,1,1,0.311267,0.622246,NaN,16765,1188
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,-225,Laborers,1.0,2,2,NaN,0.555912,0.729567,19046,225
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,-3039,Laborers,2.0,2,2,NaN,0.650442,NaN,19005,3039
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,-3038,Core staff,1.0,2,2,NaN,0.322738,NaN,19932,3038


In [9]:
cat_cols = [ 
    'NAME_CONTRACT_TYPE',
    'CODE_GENDER',
    'FLAG_OWN_CAR',
    'FLAG_OWN_REALTY', 
    'OCCUPATION_TYPE', 
    'NAME_EDUCATION_TYPE',
    'NAME_FAMILY_STATUS',
    'NAME_HOUSING_TYPE'
]

In [10]:
num_cols = [
    'TARGET',
    'CNT_CHILDREN',
    'AMT_INCOME_TOTAL',
    'AMT_CREDIT',
    'AMT_ANNUITY',
    'AMT_GOODS_PRICE',
    'REGION_POPULATION_RELATIVE',
    'TIME_EMP',
    'AGE',
    'EXT_SOURCE_1',
    'EXT_SOURCE_2',
    'EXT_SOURCE_3',
    'REGION_RATING_CLIENT',
    'REGION_RATING_CLIENT_W_CITY',
    'CNT_FAM_MEMBERS'
]

In [11]:
# Figure out which categorical columns have missing values
for col in cat_cols:
    print(col, len(df.loc[df[col].isna()]))

NAME_CONTRACT_TYPE 0
CODE_GENDER 0
FLAG_OWN_CAR 0
FLAG_OWN_REALTY 0
OCCUPATION_TYPE 96391
NAME_EDUCATION_TYPE 0
NAME_FAMILY_STATUS 0
NAME_HOUSING_TYPE 0


In [12]:
# Figure out which numerical columns have missing values
for col in num_cols:
    print(col, len(df.loc[df[col].isna()]))

TARGET 0
CNT_CHILDREN 0
AMT_INCOME_TOTAL 0
AMT_CREDIT 0
AMT_ANNUITY 12
AMT_GOODS_PRICE 278
REGION_POPULATION_RELATIVE 0
TIME_EMP 0
AGE 0
EXT_SOURCE_1 173378
EXT_SOURCE_2 660
EXT_SOURCE_3 60965
REGION_RATING_CLIENT 0
REGION_RATING_CLIENT_W_CITY 0
CNT_FAM_MEMBERS 2


In [13]:
# Remove occupation type for baseline model
del df['OCCUPATION_TYPE']

In [14]:
# Impute median value for AMT_GOODS_PRICE
df['AMT_GOODS_PRICE'].loc[df['AMT_GOODS_PRICE'].isna()] = 450000

/Users/MattByrd/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [15]:
# Impute mean values for EXT_SOURCE_1 - EXT_SOURCE_3
df['EXT_SOURCE_1'].loc[df['EXT_SOURCE_1'].isna()] = 0.5067977751549919
df['EXT_SOURCE_2'].loc[df['EXT_SOURCE_2'].isna()] = 0.5662
df['EXT_SOURCE_3'].loc[df['EXT_SOURCE_3'].isna()] = 0.5370699579791587

/Users/MattByrd/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [16]:
# Remove rows with XNA as gender
df = df[df.CODE_GENDER != "XNA"]

In [17]:
# Drop missing rows for AMT_ANNUITY and CNT_FAM_MEMBERS
df = df.dropna(subset = ['AMT_ANNUITY', 'CNT_FAM_MEMBERS'])

In [18]:
# Rerun above code to look for any remaining missing values
cat_cols = [ 
    'NAME_CONTRACT_TYPE',
    'CODE_GENDER',
    'FLAG_OWN_CAR',
    'FLAG_OWN_REALTY', 
    #'OCCUPATION_TYPE', <--- this has been removed
    'NAME_EDUCATION_TYPE',
    'NAME_FAMILY_STATUS',
    'NAME_HOUSING_TYPE'
]

In [19]:
for col in cat_cols:
    print(col, len(df.loc[df[col].isna()]))

NAME_CONTRACT_TYPE 0
CODE_GENDER 0
FLAG_OWN_CAR 0
FLAG_OWN_REALTY 0
NAME_EDUCATION_TYPE 0
NAME_FAMILY_STATUS 0
NAME_HOUSING_TYPE 0


In [20]:
for col in num_cols:
    print(col, len(df.loc[df[col].isna()]))

TARGET 0
CNT_CHILDREN 0
AMT_INCOME_TOTAL 0
AMT_CREDIT 0
AMT_ANNUITY 0
AMT_GOODS_PRICE 0
REGION_POPULATION_RELATIVE 0
TIME_EMP 0
AGE 0
EXT_SOURCE_1 0
EXT_SOURCE_2 0
EXT_SOURCE_3 0
REGION_RATING_CLIENT 0
REGION_RATING_CLIENT_W_CITY 0
CNT_FAM_MEMBERS 0


In [21]:
# Label encoding for categorical variables with 2 or less unique values
df['MALE'] = df.apply(lambda r:1 if r.CODE_GENDER == 'M' else 0, axis=1)
df['FEMALE'] = df.apply(lambda r:1 if r.CODE_GENDER == 'F' else 0, axis=1)
df['REVOLVING_LOAN'] = df.apply(lambda r:1 if r.NAME_CONTRACT_TYPE == 'Revolving loans' else 0, axis=1)
df['CASH_LOAN'] = df.apply(lambda r:1 if r.NAME_CONTRACT_TYPE == 'Cash loans' else 0, axis=1)
df['OWNS_CAR'] = df.apply(lambda r:1 if r.FLAG_OWN_CAR == 'Y' else 0, axis=1)
df['NO_CAR'] = df.apply(lambda r:1 if r.FLAG_OWN_CAR == 'N' else 0, axis=1)
df['OWNS_REALTY'] = df.apply(lambda r:1 if r.FLAG_OWN_REALTY == 'Y' else 0, axis=1)
df['NO_REALTY'] = df.apply(lambda r:1 if r.FLAG_OWN_REALTY == 'N' else 0, axis=1)

In [22]:
# Remove old (not label-encoded) variables
del df['FLAG_OWN_CAR']
del df['FLAG_OWN_REALTY']
del df['CODE_GENDER']
del df['NAME_CONTRACT_TYPE']

## Model Construction

In [23]:
X = df.copy()

In [24]:
# Remove Target and unique identifier column from training set
del X['TARGET']
del X['SK_ID_CURR']

In [27]:
X.head()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,AGE,TIME_EMP,MALE,FEMALE,REVOLVING_LOAN,CASH_LOAN,OWNS_CAR,NO_CAR,OWNS_REALTY,NO_REALTY
0,0,202500.0,406597.5,24700.5,351000.0,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,...,9461,637,1,0,0,1,0,1,1,0
1,0,270000.0,1293502.5,35698.5,1129500.0,Higher education,Married,House / apartment,0.003541,-16765,...,16765,1188,0,1,0,1,0,1,0,1
2,0,67500.0,135000.0,6750.0,135000.0,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,...,19046,225,1,0,1,0,1,0,1,0
3,0,135000.0,312682.5,29686.5,297000.0,Secondary / secondary special,Civil marriage,House / apartment,0.008019,-19005,...,19005,3039,0,1,0,1,0,1,1,0
4,0,121500.0,513000.0,21865.5,513000.0,Secondary / secondary special,Single / not married,House / apartment,0.028663,-19932,...,19932,3038,1,0,0,1,0,1,1,0


In [28]:
y = df[['TARGET']]

In [29]:
y.head(3)

,TARGET
0,1
1,0
2,0


In [30]:
# Use pd get_dummies to one-hot encode the rest of the categorical variables
X_encoded = pd.get_dummies(X)

In [31]:
X_encoded.head(3)

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,...,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents
0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,1.0,2,...,0,0,1,0,0,1,0,0,0,0
1,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,2.0,1,...,1,0,0,0,0,1,0,0,0,0
2,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,1.0,2,...,0,0,1,0,0,1,0,0,0,0


In [32]:
# Train, Test, Split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, random_state=42)

In [33]:
# Use MinMax Scaler to scale the numerical columns
scaler = MinMaxScaler(feature_range = (0, 1))

In [34]:
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [35]:
# Build a Random Forest Classifier
random_forest = RandomForestClassifier(n_estimators = 100, random_state = 50, verbose = 1, n_jobs = -1)
random_forest.fit(X_train, y_train)

/Users/MattByrd/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   19.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   44.8s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=50, verbose=1, warm_start=False)

In [36]:
# Use the classifier to make predictions on the test set from train, test, split
predictions = random_forest.predict_proba(X_test)[:, 1]
# Get the ROC_AUC score from the predictions
score = roc_auc_score(y_test, predictions)
score

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.5s finished


0.7177013912635987

### Above is the baseline score
#### TODO: Clean up this notebook and create a visualization dashboard in Tableau
#### If there's time, we can try to do a little more feature engineering to try to improve this score

### Vis Ideas
1) Correlation Matrix
2) Age and Percentage of defaulters in each Age
3) Ages binned (5 year age groups)
4) Occupation type and percentage of defaulters by Occupation type
5) EXT_SOURCE 1, 2, 3 visualizations?